List Growth Report (Lead Gen Report)

Integration:Simon Snowflake Purpose:Other

In [ ]:
/*
  The purpose of this report is to track segment sizes for 6 key audiences. 
  Each of these 6 audiences is connected to a tracking flow with a 'Do Nothing' action.
  These segments were provided by Monique Doucette (Casper).
*/

/* First, count the total adds for each tracking flow */
with flow_adds as (
select
  CASE
    WHEN campaign_id = '554458' THEN 'Active Customers' 
    WHEN campaign_id = '554468' THEN 'Inactive Customers'
    WHEN campaign_id = '554474' THEN 'Unsubscribed Customers'
    WHEN campaign_id = '554462' THEN 'Active Leads'
    WHEN campaign_id = '554466' THEN 'Inactive Leads'
    WHEN campaign_id = '554471' THEN 'Unsubscribed Leads'
  ELSE NULL
  END as Audience,
  count(*) as Segment_Size,
  campaign_id as Flow_ID,
  CURRENT_TIMESTAMP() as Report_Date,
  --define order for how to sort final output
  CASE
    WHEN campaign_id = '554458' THEN 1
    WHEN campaign_id = '554468' THEN 2
    WHEN campaign_id = '554474' THEN 3
    WHEN campaign_id = '554462' THEN 4
    WHEN campaign_id = '554466' THEN 5
    WHEN campaign_id = '554471' THEN 6
  ELSE NULL
  END as Display_Order
from prod.FLOW_ACTIONS__LATEST
where campaign_id in('554458','554468','554474','554462','554466','554471')  --Tracking Flow IDs
  and action = 'add'
and to_date(to_timestamp(ts)) <=  to_date(CURRENT_TIMESTAMP()) --Consider all adds up to report date
GROUP BY CAMPAIGN_ID, action
),

/* Next, count the removes for these same flows */
flow_removes as(
select
  CASE
    WHEN campaign_id = '554458' THEN 'Active Customers' 
    WHEN campaign_id = '554468' THEN 'Inactive Customers'
    WHEN campaign_id = '554474' THEN 'Unsubscribed Customers'
    WHEN campaign_id = '554462' THEN 'Active Leads'
    WHEN campaign_id = '554466' THEN 'Inactive Leads'
    WHEN campaign_id = '554471' THEN 'Unsubscribed Leads'
  ELSE NULL
  END as Audience,
  count(*) as Segment_Size,
  campaign_id as Flow_ID,
  CURRENT_TIMESTAMP() as Report_Date
from prod.FLOW_ACTIONS__LATEST
where campaign_id in('554458','554468','554474','554462','554466','554471') and action = 'remove'
and to_date(to_timestamp(ts)) <=  to_date(CURRENT_TIMESTAMP())
GROUP BY CAMPAIGN_ID, action
)

/* Lastly, join together adds and removes for each flow to calculate current segment size = Adds - Removes */
SELECT a.audience,
  a.segment_size - r.segment_size as segment_size,
  a.flow_id,
  a.report_date
FROM flow_adds a
LEFT JOIN flow_removes r
  on a.flow_id=r.flow_id
order by a.display_order

